In [6]:
import pyodbc, pandas as pd, openpyxl, smtplib, pymssql

In [ ]:
import json


# Load secrets
with open('secrets.json') as f:
    secrets = json.load(f)

conn = pyodbc.connect(conn_str)

query1 = "SELECT * FROM dbo.OrderInformationQuery"
query2 = "SELECT * FROM dbo.ProductInformationQuery"


conn = pyodbc.connect(conn_str)
df1 = pd.read_sql(query1, conn)
df2 = pd.read_sql(query2, conn)
# print(df1.head())
# print("----------")
# print(df2.head())
save_dir = "Output_CSV"

df1.to_csv(f"{save_dir}/OrderInformationQuery.csv", index=False)
df2.to_csv(f"{save_dir}/ProductInformationQuery.csv", index=False)


C:\Users\AlanC\AppData\Local\Temp\ipykernel_33152\1366716150.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql(query1, conn)
C:\Users\AlanC\AppData\Local\Temp\ipykernel_33152\1366716150.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql(query2, conn)
